In [ ]:
import pandas as pd
import cobra

from cobra.io import load_model

import numpy as np
import pickle

import os.path

Ponovno preberemo model:

In [ ]:
model = cobra.io.read_sbml_model(os.path.join('models','Recon3D.xml'))

Pripravimo si slovarja `id gena` --> `ime gena` in obratno:

In [ ]:
gene_id_name = {} # seznam ID genov
gene_name_id = {} # seznam imen genov

for g in model.genes[1:]:
    gene_id_name[g.id] = g.name
    gene_name_id[g.name] = g.id

In [ ]:
list(gene_id_name.items())[:5] # prvih 5 elementov v slovarju, ki ID gena poveže z imenom

In [ ]:
list(gene_name_id.items())[:5] # prvih 5 elementov v slovarju, ki ime gena poveže z ID

Preberimo podatke, ki smo jih prej shranili:

In [ ]:
df_control = pd.read_csv(os.path.join('data','data_control.txt'))
df_kd = pd.read_csv(os.path.join('data','data_kd.txt'))

Imenom genov pripišimo še ustrezne ID-je:

In [ ]:
df_control['gene_id'] = df_control['gene'].map(lambda x: gene_name_id[x] if x in gene_name_id else "")
df_kd['gene_id'] = df_kd['gene'].map(lambda x: gene_name_id[x] if x in gene_name_id else "")

In [ ]:
df_control

Ohranimo samo tiste vnose, ki jih imamo v modelu:

In [ ]:
df_control = df_control[df_control['gene_id'] != ""]
df_kd = df_kd[df_kd['gene_id'] != ""]

In [ ]:
len(df_control), len(df_kd)

In [ ]:
df_control.describe()

In [ ]:
df_kd.describe()

### Integracija transkriptomskih podatkov v model

Uporabili bomo algoritem CORDA:

* [https://resendislab.github.io/corda/](https://resendislab.github.io/corda/)
* [https://github.com/resendislab/corda](https://github.com/resendislab/corda)

Funkcija za rekonstrukcijo kot argument sprejme model in preslikavo `confidence`, ki za vsako reakcijo poda mero zaupanja, da je vsebovana v modelu:
```
Allowed confidence values
            are -1 (absent/do not include), 0 (unknown), 1 (low confidence),
            2 (medium confidence) and 3 (high confidence).
```

`confidence` si lahko pripravimo z uporabo CORDA funkcije `reaction_confidence`, ki preslika stopnje izražanja genov (po enaki lestvici kot zgoraj) v stopnje aktivnosti reakcij z uporabo pravil GPR.


Dodatno lahko nastavimo zmožnost modela, da producira biomaso:
```
met_prod (Optional[list]): Additional metabolic targets that have to be
            achieved by the model. Can be a single object or list of objects.
            List elements can be given in various forms:
            (1) A string naming a metabolite in the model will ensure that the
            metabolite can be produced.
            For instance: met_prod = ["pi_c", "atp_c"]
            (2) A dictionary of metabolite -> int will define an irreversible
            reaction that must be able to carry flux.
            For instance: met_prod = {"adp_c": -1, "pi_c": -1, "atp_c": 1}
            (3) A string representation of a reversible or irreversible
            reaction that must be able to carry flux.
            For instance: met_prod = "adp_c + pi_c -> atp_c"

```
Primer priprave niza `met_prod`:
```Python
met_prod = model.reactions.BIOMASS_maintenance.build_reaction_string()
met_prod = met_prod.replace("-->", "->")
```


In [ ]:
biomass = model.reactions.BIOMASS_maintenance.build_reaction_string().replace("-->", "->")
biomass

### Preslikava izražanja genov v prisotnosti reakcij

In [ ]:
from corda import reaction_confidence
from corda import CORDA

Pomagali si bomo s funkcijo, ki bo preslikala izražanje genov v vrednosti -1, 0, 1, 2 in 3 preko vnaprej definiranih pragov.

In [ ]:
def val_to_score(val, thr0, thr1, thr2, thr3):
    if val < thr0:
        return -1
    elif val < thr1:
        return 0
    elif val < thr2:
        return 1
    elif val < thr3:
        return 2
    else:
        return 3

#### Parametri rekonstrukcije
Pragove bomo definirali preko odstotne aktivnosti genov v vzrocu. Obstaja veliko različnih strategij, kako se tega problema lotiti. Nekaj več o tem si lahko preberete v [članku](https://doi.org/10.1371/journal.pcbi.1007185).

In [ ]:
require_biomass = True
perc0 = 25 # below this ... inactive
perc1 = 50 # else, below this ... not sure
perc2 = 75 # else, below this ...  low confidence
perc3 = 95 # else, below this ... medium confidence
# else high confidence (above perc3)

In [ ]:
folder = os.path.join('models','biomass') if require_biomass else os.path.join('models','no_biomass')
folder

### Kontrola
Za vsako skupino bomo določili lastne pragove glede na aktivnosti genov:

In [ ]:
thr0 = np.percentile(df_control.value, perc0)
thr1 = np.percentile(df_control.value, perc1)
thr2 = np.percentile(df_control.value, perc2)
thr3 = np.percentile(df_control.value, perc3)

Najprej preslikajmo intenziteto izražanja genov v skupine:

In [ ]:
#gene_conf_control = {g_id:-1 if val < thr0 else 1 if val < thr1 else 2 if val < thr2 else 3 for (g_id, val) in zip(df_control['gene_id'], df_control['value'])}
gene_conf_control = {g_id : val_to_score(val, thr0, thr1, thr2, thr3) for (g_id, val) in zip(df_control['gene_id'], df_control['value'])}

Potem poženimo integracijo z algoritmom:

**POZOR: časovno potratno! Spodnje celice lahko preskočiš in podatke prebereš iz datoteke (spodaj).**

In [ ]:
if require_biomass:
    opt_control = CORDA(model, conf_control, met_prod = biomass)
else:
    opt_control = CORDA(model, conf_control)
opt_control.build()
print(opt_control)

In [ ]:
conf_control = {}
for r in model.reactions:
    conf_control[r.id] = reaction_confidence(r, gene_conf_control)

Model pretvorimo v COBRA zapis:

In [ ]:
model_control = opt_control.cobra_model()

In mu dodajmo še biomaso:

In [ ]:
if require_biomass:
    model_control.add_reactions([model.reactions.BIOMASS_maintenance])
    model_control.objective=model.objective

Model lahko zdaj shranimo v SBML zapis:

In [ ]:
cobra.io.write_sbml_model(model_control, os.path.join(f'{folder}','model_control.xml'))

**Branje iz datoteke (če ste prejšnje korake preskočili).**

In [ ]:
model_control = cobra.io.read_sbml_model(os.path.join(f'{folder}','model_control.xml'))

In [ ]:
model_control.summary()

### Knockdown

Ponovimo podoben postopek za podatke z utišanim onkogenom:

In [ ]:
thr0 = np.percentile(df_kd.value, perc0)
thr1 = np.percentile(df_kd.value, perc1)
thr2 = np.percentile(df_kd.value, perc2)
thr3 = np.percentile(df_kd.value, perc3)

In [ ]:
gene_conf_kd = {g_id : val_to_score(val, thr0, thr1, thr2, thr3) for (g_id, val) in zip(df_kd['gene_id'], df_kd['value'])}

In [ ]:
conf_kd = {}
for r in model.reactions:
    conf_kd[r.id] = reaction_confidence(r, gene_conf_kd)

**POZOR: časovno potratno! Spodnje celice lahko preskočiš in podatke prebereš iz datoteke (spodaj).**

In [ ]:
if require_biomass:
    opt_kd = CORDA(model, conf_kd, met_prod = biomass)
else:
    opt_kd = CORDA(model, conf_kd)
opt_kd.build()
print(opt_kd)

In [ ]:
model_kd = opt_kd.cobra_model()

In [ ]:
if require_biomass:
    model_kd.add_reactions([model.reactions.BIOMASS_maintenance])
    model_kd.objective=model.objective   

In [ ]:
cobra.io.write_sbml_model(model_kd, os.path.join(f'{folder}','model_kd.xml'))

**Branje iz datoteke (če ste prejšnje korake preskočili).**

In [ ]:
model_kd = cobra.io.read_sbml_model(os.path.join(f'{folder}','model_kd.xml'))

In [ ]:
model_kd.summary()